# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
city_df = pd.read_csv('output_data/output.csv')
city_df.head()


,Unnamed: 0,City,Lat,Lng,temp,Cloudiness,Country,Date,Humidity,Max Temp,Wind Speed
0,0,boa vista,2.82,-60.67,80.6,75,BR,1592438482,83,80.6,2.24
1,1,saldanha,-33.01,17.94,50.0,11,ZA,1592438644,71,50.0,3.36
2,2,jamestown,42.10,-79.24,78.8,1,US,1592438529,41,78.8,11.41
3,3,albany,42.60,-73.97,82.0,0,US,1592438644,47,82.0,1.88
4,4,hilo,19.73,-155.09,84.2,1,US,1592438640,54,84.2,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [9]:
# Store latitude and longitude in locations
locations = city_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
Humidity = city_df["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [18]:
#Create New DataFrame to narrow down the cities to fit weather conditions
fit_weather_df = city_df.loc[city_df['Max Temp'] < 70]
fit_weather_df = fit_weather_df.loc[fit_weather_df['Wind Speed'] < 13]
fit_weather_df

,Unnamed: 0,City,Lat,Lng,temp,Cloudiness,Country,Date,Humidity,Max Temp,Wind Speed
1,1,saldanha,-33.01,17.94,50.00,11,ZA,1592438644,71,50.00,3.36
6,6,barrow,71.29,-156.79,42.80,1,US,1592438645,70,42.80,12.75
9,9,ushuaia,-54.80,-68.30,35.60,57,AR,1592438529,80,35.60,6.93
10,10,mount gambier,-37.83,140.77,48.20,66,AU,1592438646,81,48.20,12.75
13,13,san martin,-33.08,-68.47,50.00,38,AR,1592438646,57,50.00,3.36
...,...,...,...,...,...,...,...,...,...,...,...
534,534,puerto madryn,-42.77,-65.04,53.15,97,AR,1592438726,50,53.15,9.86
535,535,vitim,59.45,112.56,63.50,68,RU,1592438726,61,63.50,1.41
539,539,ayan,56.45,138.17,42.67,98,RU,1592438726,96,42.67,4.83
541,541,kalmar,56.66,16.36,52.00,4,SE,1592438727,100,52.00,2.24


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
